## Structured Output: Guaranteed Responses

Structured output refers to the ability of the `ChatCompletions` API to return responses in a predefined format, such as a JSON object or a Pydantic Model. This is particularly useful when you need the model to adhere to a specific schema for downstream processing or integration with other systems. By defining the expected structure, you can ensure the response is validated and parsed into a predictable format. 

Key Features of Structured Outputs

1. Customizable Response Format
    - You can specify the expected structure of the response using the response_format parameter.
    - This can be defined as either a JSON schema or a Pydantic model, depending on your requirements.
2. Using JSON Schema with create:
    - The `chat.completions.create` method allows you to provide a JSON schema via the `response_format` parameter.
    - This guides the model to generate responses in the desired structure without requiring Python-based schema definitions.
3. Using Pydantic Models with parse
    - The `chat.completions.parse` method supports validation and parsing using Pydantic models.
    - This is ideal for scenarios where you need Python-based schema definitions and strict adherance to the structure.

### Why would we need this?

In Part 2, we saw that we could ask the model to format its response as a JSON object using a prompt. This is a big step up from unstructured text, but it's still fundamentally a suggestion, not a guarantee.

Let's see what happens when we try to extract information from a simple sentence and ask for a JSON response. Our goal is to get a clean JSON object that we can immediately load and use in our Python code.

In [3]:
# Let's set up our client first
from openai import OpenAI
import json

# Read the API_KEY
with open('API_KEY.txt', 'r') as file:
    API_KEY = file.read()

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY, # Make sure to use your key
)

In [5]:
# Our prompt asks for a specific JSON structure
prompt = """
Extract the grammatical components from the following sentence.
Please respond with ONLY a JSON object with the keys 'subject', 'verb', and 'object'.

Sentence: The researcher analyzed the data.
"""

response = client.chat.completions.create(
    model="mistralai/mistral-small-3.2-24b-instruct:free",
    messages=[
        {"role": "user", "content": prompt}
    ],
)

raw_content = response.choices[0].message.content
print("--- Raw LLM Output ---")
print(raw_content)

--- Raw LLM Output ---
```json
{
  "subject": "The researcher",
  "verb": "analyzed",
  "object": "the data"
}
```


Now, let's try to parse this output as JSON, which is what you would do in any real-world data pipeline.

In [8]:
try:
    parsed_json = json.loads(raw_content)
    print("--- Successfully Parsed JSON ---")
    print(parsed_json)
except json.JSONDecodeError as e:
    print(f"--- FAILED to Parse JSON ---")
    print(f"Error: {e}")

--- FAILED to Parse JSON ---
Error: Expecting value: line 1 column 1 (char 0)


### Why Did That Fail?
🔔 Question: Look at the raw output from the LLM. Why did the json.loads() function crash?

You'll likely see one of these common failure modes:

1. Conversational Chit-Chat: The model might wrap the JSON in helpful text, like:

    >"Sure, here is the JSON you requested: { ... }"
    <br><br>
    "{ ... } I hope this helps!"
2. Markdown Code Blocks: 
    <br><br>
    Often, the model will format the JSON within a markdown block, which looks like this:
    ```json
    {
        "subject": "The researcher",
        "verb": "analyzed",
        "object": "the data"
    }
    ```
3. Inconsistent Formatting
    <br><br>Sometimes it might use single quotes instead of double quotes, which is invalid in JSON. Or forgot to properly close a bracket. Since the output is non deterministic there's no way to always guarantee proper formatting from just a text 

While you could write a bunch of string manipulation code `.strip(), .replace(), etc.` to clean this up, that's brittle and unreliable. If you're processing 10,000 documents, you need a system that works every single time. Your research pipeline will break if even a small percentage of responses are not in the exact format you expect.

This is the core problem that guaranteed structured output solves. We need to move from prompting for a format to demanding it.

This is where the response_format parameter and Pydantic models come in. They provide a strict "contract" with the API, ensuring the response is not just close to what you want, but is a perfectly formatted, validated, and ready-to-use Python object.

### A Step in the Right Direction: `response_format`
Okay, that last attempt was messy. Constantly cleaning up the model's output is not a scalable solution.

Fortunately, the `ChatCompletions` API has a built-in parameter to help with this: `response_format`. By setting response_format={"type": "json_object"}, we can instruct the model to enable "JSON mode."

When JSON mode is enabled, the model is constrained to only generate strings that can be parsed into valid JSON. This eliminates the problems of conversational wrappers and markdown formatting.

Let's try our previous example again, but this time using JSON mode.

In [12]:
# Our prompt is now simpler. We don't need to beg for JSON in the text itself.
prompt = "Extract the subject, verb, and object from this sentence: The researcher analyzed the data."

response = client.chat.completions.create(
    model="nvidia/nemotron-nano-9b-v2:free",
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": prompt}
    ],
    response_format={"type": "json_object"} # This is the key parameter!
)

raw_content = response.choices[0].message.content
print("--- Raw LLM Output ---")
print(raw_content)

# Now, let's try to parse it again.
try:
    parsed_json = json.loads(raw_content)
    print("\\n--- Successfully Parsed JSON ---")
    print(parsed_json)
    print(f"Subject: {parsed_json.get('subject')}")
except json.JSONDecodeError as e:
    print(f"\\n--- FAILED to Parse JSON ---")
    print(f"Error: {e}")

--- Raw LLM Output ---
{
  "subject": "The researcher",
  "verb": "analyzed",
  "object": "the data"
}
\n--- Successfully Parsed JSON ---
{'subject': 'The researcher', 'verb': 'analyzed', 'object': 'the data'}
Subject: The researcher


### The New Problem: Correct Syntax, Wrong Schema
This is much better! We got a clean, parsable JSON object directly from the API.

But we're not out of the woods yet. JSON mode guarantees syntactic validity (the output is valid JSON), but it does not guarantee semantic validity (the output matches the schema or structure we actually need).

The model is still free to:
- Invent new keys ("theme": "research").
- Forget required keys ("object").
- Return the wrong data type (e.g., a number instead of a string).

Let's design a prompt that might confuse the model and see if we can expose this weakness. We'll ask it for a user's name and age, expecting specific keys and data types.

In [ ]:
# We expect a JSON object like: {"user_name": str, "user_age": int}

confusing_prompt = "Extract the user's details. The user, Alex, is 25 years old."

response = client.chat.completions.create(
    model="nvidia/nemotron-nano-9b-v2:free",
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": confusing_prompt}
    ],
    response_format={"type": "json_object"}
)

raw_content = response.choices[0].message.content
print("--- Raw LLM Output from Confusing Prompt ---")
print(raw_content)

# Let's check if our expected keys exist and have the correct types
try:
    data = json.loads(raw_content)
    user_name = data['user_name']
    user_age = data['user_age']

    if not isinstance(user_age, int):
        print("\\n--- VALIDATION FAILED: 'user_age' should be an integer! ---")
    else:
        print("\\n--- Validation Passed ---")
        print(f"{user_name} is {user_age} years old.")

except KeyError as e:
    print(f"\\n--- VALIDATION FAILED: Missing key in response: {e} ---")
except json.JSONDecodeError as e:
     print(f"\\n--- FAILED to Parse JSON ---")
     print(f"Error: {e}")

--- Raw LLM Output from Confusing Prompt ---
{
  "user_name": "Alex",
  "user_age": 25
}
\n--- Validation Passed ---
Alex is 25 years old.


🔔 Question: Run the cell above a few times. Do you always get the keys user_name and user_age? Does the model sometimes use different keys like "name" or "age"? Does it ever return the age as a string ("25") instead of an integer (25)?

This inconsistency is the final hurdle. For a truly robust data pipeline, we need to guarantee not just the format (JSON) but also the schema (the exact keys and data types). Again even if you don't see the responses fail in this small sample, when running a script 10,000 times. You don't want there ever to be mistakes or it could cause your whole workflow to fail. 

This is what Pydantic models are for, and it's what we'll cover next.

### This leads us back to: Structured Output

We've seen that prompting for JSON is unreliable and that `response_format={"type": "json_object"}` only guarantees valid syntax, not the correct content or structure. We need a way to define a strict schema—a blueprint for our data—and force the model's output to conform to it.

This is precisely what the `Pydantic` library was built for.

`Pydantic` allows you to define a data structure as a Python class. When we combine this with the special `.parse()` method from the client, we are no longer just hoping for the right output; we are guaranteeing it.

#### Your First Pydantic Model
Let's solve our simple sentence-parsing problem, this time with a guarantee of success.

First, you'll need to install Pydantic:



In [ ]:
pip install pydantic

Now, let's define our "contract" as a Python class. It looks simple, but it's incredibly powerful. Each attribute defines an expected key and its required data type.

In [20]:
from pydantic import BaseModel

# First, define the structure of your desired output as a Pydantic class.
# This class IS the schema.
class ParsedSentence(BaseModel):
    subject: str
    verb: str
    obj: str # 'obj' is the key, and it MUST be a string.

Instead of using client.chat.completions.create, we will use client.chat.completions.parse. This special method is designed to work with Pydantic models.

In [ ]:
# The prompt is dead simple. All the complexity is in our Pydantic model.
prompt = "The researcher analyzed the data."

# We use .parse() and pass our Pydantic class to the response_format parameter.
response = client.chat.completions.parse(
    model="mistralai/mistral-small-3.2-24b-instruct:free",
    messages=[
        {"role": "user", "content": prompt}
    ],
    response_format=ParsedSentence, # This is the magic!
)

In [30]:
from pprint import pprint

In [31]:
# The response object is slightly different from before.
print("--- Raw Response Object ---")
pprint(response.model_dump())

--- Raw Response Object ---
{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'annotations': None,
                          'audio': None,
                          'content': '{ "subject": "researcher", "verb": '
                                     '"analyzed", "obj": "data" }',
                          'function_call': None,
                          'parsed': {'obj': 'data',
                                     'subject': 'researcher',
                                     'verb': 'analyzed'},
                          'reasoning': None,
                          'refusal': None,
                          'role': 'assistant',
                          'tool_calls': None},
              'native_finish_reason': 'stop'}],
 'created': 1759979401,
 'id': 'gen-1759979401-ws7G92DRzanNZr5TuBHN',
 'model': 'mistralai/mistral-small-3.2-24b-instruct:free',
 'object': 'chat.completion',
 'provider': 'Chutes',
 'service_t

Look at the output. The message object now contains a special field: parsed. This is our Pydantic object, fully validated and ready to use.

🔔 Question: What do you notice that is special about this parsed field?

In [ ]:
# Pay attention to the type of the parsed field.
type(response.choices[0].message.parsed) 

__main__.ParsedSentence

In [34]:
# No more json.loads() or try/except blocks needed!
# We can directly access our validated data.
parsed_data = response.choices[0].message.parsed

print("\\n--- Accessing the Parsed Data ---")
print(f"Subject: {parsed_data.subject}")
print(f"Verb: {parsed_data.verb}")
print(f"Object: {parsed_data.obj}")

\n--- Accessing the Parsed Data ---
Subject: researcher
Verb: analyzed
Object: data


This is the gold standard for reliable data extraction. By defining a Pydantic BaseModel, you create a robust and unbreakable contract with the LLM, ensuring that every response you get is structured exactly the way you need it to be for your downstream analysis.

---

Now it's your turn to build some Pydantic models. We'll work through a series of short challenges, each designed to teach you a core feature. The goal is to get your hands dirty and build confidence before we tackle a larger, more complex problem.

For each challenge, you will:

Define the Pydantic BaseModel that matches the required schema.

Make the API call using client.chat.completions.parse and your new model.

Print the result to verify that your model worked correctly.

### Challenge 1: Basic Data Types
**Goal**: Extract the name (string), age (integer), and student status (boolean) from the text below.

In [ ]:
from pydantic import BaseModel

# The text to parse
prompt_text = "Meet David Chen. He is 42 years old and works as an engineer, so he is not a student."

# 1. DEFINE YOUR SCHEMA HERE
# Create a Pydantic class called 'PersonProfile' with the following fields:
# - name: str
# - age: int
# - is_student: bool
class PersonProfile(BaseModel):
    name: str
    age: int
    is_student: bool


# 2. MAKE THE API CALL
# Use client.chat.completions.parse with your PersonProfile model.

response = client.chat.completions.parse(
    model="mistralai/mistral-small-3.2-24b-instruct:free",
    messages=[
        {"role": "user", "content": prompt_text}
    ],
    response_format=PersonProfile, # Use your Pydantic model here
)

# 3. PRINT THE RESULT
parsed_result = response.choices[0].message.parsed
print(parsed_result)
print(f"The type of 'age' is: {type(parsed_result.age)}")

Key Concept: Pydantic handles the conversion from the model's text output (e.g., "42") to the correct Python data type (the integer 42).

### Challenge 2: Handling Lists of Strings

**Goal**: Extract the name of the committee and a list of all its members.

In [ ]:
from pydantic import BaseModel
from typing import List

# The text to parse
prompt_text = "The budget committee is composed of several members: Maria, David, and Susan."

# 1. DEFINE YOUR SCHEMA HERE
# Create a Pydantic class called 'Committee' with the following fields:
# - committee_name: str
# - members: List[str]

class Committee(BaseModel):
    committee_name: str
    members: List[str]

# 2. MAKE THE API CALL
response = client.chat.completions.parse(
    model="mistralai/mistral-small-3.2-24b-instruct:free",
    messages=[
        {"role": "user", "content": prompt_text}
    ],
    response_format=Committee,  # Use your Pydantic model here
)

# 3. PRINT THE RESULT
parsed_result = response.choices[0].message.parsed
print(f"Committee: {parsed_result.committee_name}")
print(f"Members: {parsed_result.members}")

Key Concept: Use List[<type>] (e.g., List[str]) to extract a list of items.

### Challenge 3: Optional Fields
**Goal**: Extract a product's name and its rating. The rating might not always be present.

In [ ]:
from pydantic import BaseModel
from typing import Optional

# Text 1: Contains a rating
prompt_text_1 = "The new ACME Anvil 2.0 is a fantastic product. I'd give it a 5 out of 5 stars."

# Text 2: Does NOT contain a rating
prompt_text_2 = "We have just received a shipment of the ACME Anvil 2.0."

# 1. DEFINE YOUR SCHEMA HERE
# Create a Pydantic class called 'Product' with the following fields:
# - product_name: str
# - rating: Optional[int]  # This field can be an integer or None

class Product(BaseModel):
    product_name: str
    rating: Optional[int]  # This field can be an integer or None

# 2. MAKE THE API CALL (Try it with both prompt_text_1 and prompt_text_2!)
response_1 = client.chat.completions.parse(
    model="mistralai/mistral-small-3.2-24b-instruct:free",
    messages=[
        {"role": "user", "content": prompt_text_1}
    ],
    response_format=Product,
)

response_2 = client.chat.completions.parse(
    model="mistralai/mistral-small-3.2-24b-instruct:free",
    messages=[
        {"role": "user", "content": prompt_text_2}
    ], 
    response_format=Product,
)
   
# 3. PRINT THE RESULT
parsed_result_1 = response_1.choices[0].message.parsed
print(parsed_result_1.model_dump())

parsed_result_2 = response_2.choices[0].message.parsed
print(parsed_result_2.model_dump())

Key Concept: Use Optional[<type>] for data that might be missing in the source text. This prevents your code from crashing.

### Challenge 4: Nesting Models
Goal: Extract information about a report, including a nested object for the author.

In [ ]:
from pydantic import BaseModel

# The text to parse
prompt_text = "The 2023 annual report on climate change was written by Dr. Eleanor Vance."

# 1. DEFINE YOUR SCHEMAS HERE
# First, create an 'Author' model with fields:
# - first_name: str
# - last_name: str

# class Author(BaseModel):
class Author(BaseModel):
    first_name: str
    last_name: str

# Next, create a 'Report' model with fields:
# - title: str
# - year: int
# - author: Author  # Use the Author model as the type here!

class Report(BaseModel):
    title: str
    year: int
    author: Author


# 2. MAKE THE API CALL
response = client.chat.completions.parse(
    model="mistralai/mistral-small-3.2-24b-instruct:free",
    messages=[
        {"role": "user", "content": prompt_text}
    ],
    response_format=Report,
)

# 3. PRINT THE RESULT
parsed_result = response.choices[0].message.parsed
print(f"Report: '{parsed_result.title}' ({parsed_result.year})")
print(f"Author: {parsed_result.author.first_name} {parsed_result.author.last_name}")

Key Concept: You can create complex schemas by nesting models within each other, which is essential for representing real-world data structures.

---

## Part 3 Wrap-Up: From Suggestion to Guarantee
Congratulations! You have just mastered the single most important technique for building reliable, scalable applications with Large Language Models.

Let's quickly recap the journey you took in this section:

1. **The Problem with Prompts**: You started by seeing why simply asking for JSON in a prompt is unreliable. The model can add conversational text or markdown, breaking any automated data processing pipeline.
2. **The Limits of JSON Mode**: You learned that response_format={"type": "json_object"} is a step up, guaranteeing valid JSON syntax, but it offers no protection against the model inventing keys or using the wrong data types—it doesn't guarantee the schema.
3. **The Power of Pydantic**: Finally, you learned how to use Pydantic models with the .parse() method to create an unbreakable contract with the LLM. Through the challenges, you proved you can now reliably extract:
    - Correct data types (strings, integers, booleans)
    - Lists of items
    - Optional and nested data

You've moved from suggesting a format to guaranteeing it. This skill is the cornerstone of any serious research or production workflow that uses LLMs for data extraction, classification, or analysis.

Now that you can reliably get structured data out of a model, what's next? In Part 4, we'll explore how to give the model abilities to interact with the outside world using Tool Calling.